In [1]:
from sklearn.externals import joblib
from my_data_preparation_kit import normalize # ユーザ発話に対する処理(コーパスからユーザ発話を取得する機能は不要)
from my_data_preparation_kit import word_table_json_name, synonym_table_json_name, load_json_as_dict # JSONファイル周りの処理
from my_feature_vector_creator_kit import get_all_words_in_corpus
import time
import MeCab
import random
import numpy as np

svr_model = "tuned_rbf_svr_model.pkl"
constructed_corpus_path = "dialogue_corpus.db"
pn_ja_dict_path = "../pn_ja.dic"

# data_preparation.ipynbで保存したword_tableとsynonym_tableを読み込む
w_table = load_json_as_dict(word_table_json_name)       # print(convert_dict_to_json(word_table, 4))   -> {"u\...": [], ... }
s_table = load_json_as_dict(synonym_table_json_name) # print(word_table["食べる"])                  -> [235301]

In [2]:
# 学習(チューニング)済みのSVRのモデルを読み込む
t1 = time.time()
loaded_model = joblib.load(svr_model)
t2 = time.time()
print("Finished loading learned SVR model: about {} seconds.".format(t2 - t1))

Finished loading learned SVR model: about 0.29519104957580566 seconds.


In [3]:
# 単語極性辞書の読み込み
pn_table = {}
with open(pn_ja_dict_path) as file:
    for line in file:
        line = line.split(":")
        pn_table[line[0]] = float(line[3])
print("Finished loading pn_ja.dic.")

Finished loading pn_ja.dic.


In [4]:
# コーパス内の全単語
all_words_in_corpus = get_all_words_in_corpus()
# コーパス内の全単語の重複無しバージョン（順序を保持している）
# 参考サイト：https://note.nkmk.me/python-list-unique-duplicate/
tt = time.time()
all_words_in_corpus_uniq = sorted(set(all_words_in_corpus), key = all_words_in_corpus.index)
tt2 = time.time()
print("Finished making all_words-list which removed duplication. (order of the list is kept): about {} seconds.".format(tt2 - tt))

Finished getting all words from dialogue_corpus.db: about 1.1478192806243896 seconds.
Finished making all_words-list which removed duplication. (order of the list is kept): about 17.739309549331665 seconds.


In [5]:
# コーパス内の全単語とその同義語を、対応する列にまとめたリスト
# 対応する単語IDが無い or 同義語が無い場合は、その単語自体のみを含んだリストが格納される
t0 = time.time()
columns = []
for word in all_words_in_corpus_uniq:
    column = [word]
    wordids = []
    if word in w_table.keys(): # wordがユーザ発話に含まれる単語ならば
        wordids = w_table[word]
    if wordids != []: # wordに対応する単語IDがあれば
        for wordid in wordids:
            # wordidに対応する同義語をcolumnに追加
            synonyms = s_table[str(wordid)]
            column.extend(synonyms)
    columns.append(column)
t01 = time.time()
print("Finished making a list about words in corpus and their synonyms: about {} seconds.".format(t01 - t0))

Finished making a list about words in corpus and their synonyms: about 0.058042287826538086 seconds.


In [6]:
# my_feature_vector_creator_kit.pyのmake_svr_learning_featureと
# ほぼ同じ手順で数値表現に直す
def make_bow_of_inputted_usertalk(inputted_usertalk):
    m = MeCab.Tagger("-Owakati")
    bow = [0] * len(columns) # 入力されたユーザ発話の特徴ベクトル
    word_pn_scores = 0 # ユーザ発話の極性スコアの記録
    words_in_inputted_talk = m.parse(inputted_usertalk).split() # ユーザ発話に含まれる単語のリスト
    for word in words_in_inputted_talk:
        # word：ユーザ発話に含まれる単語
        # columns：コーパス内の全単語とその同義語を、対応する列にまとめたリスト
        for col_i, column in enumerate(columns):
            if word in column: # wordが、対応する列にまとめておいた単語と同義語のリストに含まれていたら
                bow[col_i] = 1

        if word in pn_table.keys(): # 1つのユーザ発話内の単語の極性スコアを加算
            word_pn_scores += pn_table[word]
    else:
        # 1発話の単語について全て調べ終わったら、発話文の各単語の極性スコアの平均値を算出する
        # 小数点以下切り捨て防止のため分子にfloat()、0除算防止のため分母に+1
        word_pn_ave = float(word_pn_scores) / (len(words_in_inputted_talk) + 1)
        bow.append(word_pn_ave)

    bow = [bow] # 二次元である学習データに合わせるため
    return bow

In [7]:
scene_table = {1: "掃除をすること", 2: "食べ残しをしないこと", 3: "早く寝ること", 4: "ゲームを止めること", 5: "適度な運動をとること"}
random_num = random.randint(1, 5) # ランダムに場面を選択する
selected_scene = scene_table[random_num]
request_sentence = "" # システムの依頼文

if random_num == 1:
    request_sentence = "さあ、部屋を掃除しようよ。"
elif random_num == 2:
    request_sentence = "食べ残しは良くないよ...。"
elif random_num == 3:
    request_sentence = "早く寝ろよ。"
elif random_num == 4:
    request_sentence = "ゲームやりすぎじゃない？"
elif random_num == 5:
    request_sentence = "たまには運動しようぜ！"
print("選択された場面：{}".format(selected_scene))

選択された場面：食べ残しをしないこと


In [13]:
while True:
    # システムから発話
    print("system -> {}".format(request_sentence))
    
    # ユーザの入力発話
    inputted_usertalk = input("user -> ")
    usertalk_bow = make_bow_of_inputted_usertalk(inputted_usertalk) # ユーザ発話を数値表現に変える
    
    # ユーザの受諾度合いの推定
    user_acceptance = loaded_model.predict(usertalk_bow)
    print("ユーザ発話 = {}, 受諾度合いの予測値 = {}".format(inputted_usertalk, user_acceptance))
    print(np.round(user_acceptance))
    
    # システム感情の遷移
    
    # 応答文選択
    
    # システムの応答
    break

system -> 食べ残しは良くないよ...。
user -> 良いじゃん。
ユーザ発話 = 良いじゃん。, 受諾度合いの予測値 = [ 2.27737739]
[ 2.]
